# 一、 数据获取余票数据

# 1.几行简单代码获取某区间某日所有余票数据

## 1.1读取SHANGHAI-SUZHOU单个页面数据

首先要分析网站，怎么样才能获取数据

In [1]:
import requests
import json

In [ ]:
url = 'https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&queryDate=2016-12-29&from_station=SHH&to_station=NJH'

In [ ]:
js = requests.get(url).text

https://promotion.aliyun.com/ntms/act/globalhttps.html?spm=5176.8142029.418686.5.yBwhSy<br>
HTTPS在HTTP的基础上加入了SSL/TLS协议，依靠SSL证书来验证服务器的身份，并为客户端和服务器端之间建立“SSL加密通道”，确保用户数据在传输过程中处于加密状态，同时防止服务器被钓鱼网站假冒。

In [ ]:
js = requests.get(url,verify=False).text

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
js = requests.get(url,verify=False).text

In [ ]:
js2 = json.loads(js)

In [ ]:
data = js2['data']['datas']

In [ ]:
data

## 1.2将数据读入pandas并进行处理后保存

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(data)
df.tail()

In [ ]:
df.columns

In [ ]:
df_use = df[[u'train_no',
            u'station_train_code',
             u'from_station_name',
             u'to_station_name',
             u'start_time',
             u'swz_num',
             u'tz_num',  #特等座
             u'zy_num',
             u'ze_num',
             u'wz_num',
             ]]

In [ ]:
df_use.tail()

In [ ]:
df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
       df_use[u'station_train_code'].str.startswith('D')]
df_use.tail()

In [ ]:
df_use.replace('--',0,inplace=True) # 注意：--是指无此席位，并非指无余票
df_use.replace(u'无',0,inplace=True)

In [ ]:
# df_use.reset_index(inplace=True)
df_use.tail()

In [ ]:
df_use.to_csv('./data/TEST2016-12-29-SH-NJ.csv',encoding='utf-8')

## 1.3一个完整代码

In [ ]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")

def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                     para, verify=False).text
    return r

def to_csv(js):
    js2 = json.loads(js)
    data = js2['data']['datas']
    df = pd.DataFrame(data)
    df_use = df[[u'train_no',
            u'station_train_code',
                 u'from_station_name',
                 u'to_station_name',
                 u'start_time',
                 u'swz_num',
                 u'tz_num',
                 u'zy_num',
                 u'ze_num',
                 u'wz_num',
                 ]]
    df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    df_use[u'station_train_code'].str.startswith('D')]
    df_use.replace('--',0,inplace=True)
    df_use.replace(u'无',0,inplace=True)
#     tm = arrow.now().format('HH:mm:ss')
#     df_use['record_time'] = tm
    df_use.to_csv('./data/TEST2016-12-29-SH-NJ.csv',index=False,encoding='utf-8')
if __name__ == '__main__':
    para = 'queryDate=2016-12-29&from_station=SHH&to_station=NJH'
    js = get_pd(para)
    to_csv(js)
    print('finished')

#### 多sheet录入
>writer = pd.ExcelWriter('./data/ht2016-12-10-SH-NJ.xlsx') <br>
>df_use.to_excel(writer,'Sheet1')<br>
>df.to_excel(writer,'Sheet2') <br>
>writer.save()<br>

#  2. 类-面向对象（获取沪宁、宁沪所有站点某日所有余票数据）

```
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
# station_ls_comb = []
for i in range(0,len(hu_ning)-1):
    ls = [hu_ning[i],hu_ning[i+1]]
#     station_ls_comb.append(ls)
    print ls
# print station_ls_comb
```

In [ ]:
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
ning_hu = ['NJH','ZJH','DYH','CZH','WXH','SZH','KNH','SHH']

## 2.1进行站点之间组合

# $C^{2}_8$   

In [ ]:
from itertools import combinations
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
com = combinations(hu_ning,2)
for i in com:
    print i

## 2.2抓取特定时间所有站点数据-运行一次

In [ ]:
import requests
import pandas as pd
import json
import arrow
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")

class HighSpeed(object):
    def __init__(self,date,from_station,to_station):
        self.date = date
        self.from_station = from_station    #发车站
        self.to_station = to_station        #到达站

    
#     url = 'https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT
#     &queryDate=2016-12-29&from_station=SHH&to_station=SZH'

    def get_pd(self):
        para = 'queryDate=' + self.date + '&from_station=' + self.from_station \
               +'&to_station=' + self.to_station
        r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                         para, verify=False).text
        return r

    def to_csv(self):
        js = self.get_pd()
        js2 = json.loads(js)
        data = js2['data']['datas']
        df = pd.DataFrame(data)
        df_use = df[[u'train_no',
                u'station_train_code',
                     u'from_station_name',
                     u'to_station_name',
                     u'start_time',
                     u'swz_num',
                     u'tz_num',
                     u'zy_num',
                     u'ze_num',
                     u'wz_num',
                     ]]
        df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
        df_use[u'station_train_code'].str.startswith('D')]
        df_use.replace('--',0,inplace=True)
        df_use.replace(u'无',0,inplace=True)    
        
        tm = arrow.now().format('HH:mm')
        df_use['record_time'] = tm
        
#         df_use = df_choose[df_choose['record_time'] == df_choose['start_time']]
      
        df_use.to_csv('./data/' + self.date + 'TEST.csv',
                       mode = 'a', index = False, 
                      column = False,
                      encoding='utf-8')
         
if __name__ == '__main__':
    date = '2016-12-29'    
    hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
    com = combinations(hu_ning,2)
    for ls in com:
        hspeed = HighSpeed(date,ls[0],ls[1])
        hspeed.to_csv()
    print('finished')

# 3.按时间自动抓取余票数据

## 3.1 时间库与流程库

#### 流程控制库

In [ ]:
import schedule
import time

def job():
    print("I'm working...")

schedule.every(1).minutes.do(job)
# schedule.every().hour.do(job)
# schedule.every().day.at("14:09").do(job)
# schedule.every().monday.do(job)
# schedule.every().wednesday.at("13:15").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
import threading
import time
import schedule
import arrow

def job():
    date = arrow.now().format('mm:ss')
    print("I'm running on job",date)

def job2():
    date = arrow.now().format('mm:ss')
    print("I'm running on job2",date)

def run_threaded(job_func):
    job_thread = threading.Thread(target=job_func)
    job_thread.start()


schedule.every(10).seconds.do(run_threaded, job)
schedule.every(10).seconds.do(run_threaded, job2)



while 1:
    schedule.run_pending()
    time.sleep(1)

#### 时间转换库

In [ ]:
import arrow
tm = arrow.now()
print tm.format('HH:mm:ss')
    
tm_debefore30m = tm.replace(minutes=-30)
print tm_debefore30m

In [ ]:
tm2_debefore30m = tm.replace(minutes=-30).format('HH:mm')
print tm2_debefore30m

In [ ]:
date = arrow.now().format('YYYY-MM-DD')
date

## 3.2 每间隔一分钟抓取一次，所有站点数据-运行一次

In [ ]:
# -*- coding:utf-8 -*-

import requests
import pandas as pd
import json
import schedule
import threading
import time
import arrow
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")

class HighSpeed(object):
    def __init__(self,date,from_station,to_station,direction):
        self.date = date
        self.from_station = from_station
        self.to_station = to_station
        self.direction = direction

    def get_pd(self):
        try:
            para = 'queryDate=' + self.date + '&from_station=' + self.from_station \
                   +'&to_station=' + self.to_station
            r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                             para, verify=False).text
            return r
        except:
            fl = open('./data/log.txt','a')
            fl.write('requests error at ' + str(arrow.now().format('HH:mm'))+ "\n")
            fl.close()
            print("requests error at "+str(arrow.now().format('HH:mm')))

    def to_csv(self):
        js = self.get_pd()
        try:
            js2 = json.loads(js)
            data = js2['data']['datas']
            df = pd.DataFrame(data)
            df_use = df[[u'train_no',
                     u'station_train_code',
                         u'from_station_name',
                         u'to_station_name',
                         u'start_time',
                         u'swz_num',
                         u'tz_num',
                         u'zy_num',
                         u'ze_num',
                         u'wz_num',
                         ]]
            df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
            df_use[u'station_train_code'].str.startswith('D')]
            df_use.replace('--',0,inplace=True)
            df_use.replace(u'无',0,inplace=True)
            tm = arrow.now()
            df_use['record_date'] = tm.format('YYYY-MM-DD')
            df_use['record_time'] = tm.format('HH:mm')
            df_use.to_csv('./data/' + self.direction + "-" +self.date + '.csv',mode = 'a', index = False, header = False,encoding='utf-8')
        except:
            print 'no json data!'
        print 'I am working!'

    
        
if __name__ == '__main__':
       
    def process_hspeed():
        date = arrow.now().format('YYYY-MM-DD')    
        
        hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
        com = combinations(hu_ning,2)
        for ls_hu_ning in com:
            hspeed = HighSpeed(date,ls_hu_ning[0],ls_hu_ning[1],'HuNing')
            hspeed.to_csv()
    
    def process_hspeed2():   
        date = arrow.now().format('YYYY-MM-DD')
        ning_hu = ['NJH','ZJH','DYH','CZH','WXH','SZH','KNH','SHH']
        com = combinations(ning_hu,2)
        for ls_ning_hu in com:
            hspeed = HighSpeed(date,ls_ning_hu[0],ls_ning_hu[1],'NingHu')
            hspeed.to_csv()

    def run_threaded(job_func):
        job_thread = threading.Thread(target=job_func)
        job_thread.start()


#     def job():
#         print("I'm working...")

    schedule.every(1).minutes.do(run_threaded,process_hspeed)
    schedule.every(1).minutes.do(run_threaded,process_hspeed2)
    # schedule.every().hour.do(job)
    # schedule.every().day.at("14:09").do(job)
    # schedule.every().monday.do(job)
    # schedule.every().wednesday.at("13:15").do(job)

    while True:
        schedule.run_pending()
        time.sleep(1)


# 4.判断可用数据

In [ ]:
import pandas as pd
import arrow
df_hu_ning = pd.read_csv('./data/HuNing-2016-12-21.csv',encoding='utf-8')
df_hu_ning.tail()

In [ ]:
df_hu_ning.columns = [u'train_no',
    u'station_train_code',
                     u'from_station_name',
                     u'to_station_name',
                     u'start_time',
                     u'swz_num',
                     u'tz_num',
                     u'zy_num',
                     u'ze_num',
                     u'wz_num',
              u'record_date',
              u'record_time',]
df_hu_ning.tail()

In [ ]:
df_hu_ning[df_hu_ning['station_train_code'] == 'G1814']

In [ ]:
df_hu_ning[(df_hu_ning['station_train_code'] == 'G1814') & (df_hu_ning['record_time'] == '12:20')]

## 4.1 筛选出发车时间前30分钟的余票信息

> 上海——南京，最早发车时间为06:02，最后一个区间是	22:43  G7128  镇江至南京，<br>
> 南京——上海，最早发车时间为03:30，最后一个区间是	22:43  G7128  镇江至南京，

In [ ]:
df_hu_ning2 = df_hu_ning[(df_hu_ning['start_time'].str.contains("0[6-9]:|1[0-9]:|2[0-3]:"))]
df_hu_ning2['start_time'].sort_values(ascending=True).head()

In [ ]:
df_hu_ning2.head()

In [ ]:
time_choose = arrow.now()
print time_choose
time_before_30m = time_choose.replace(minutes=-30).format('HH:mm')
print time_before_30m

#### 小记录：开车前30分钟(test)

In [ ]:
df_hu_ning2['start_time'].head()

In [ ]:
df_hu_ning2['start_time'].tail()

In [ ]:
arrow.get('08:05', 'HH:mm').replace(minutes=-30).format('HH:mm')

In [ ]:
df_hu_ning2['start_time'].apply(lambda x: arrow.get(x, 'HH:mm').replace(minutes=-30).format('HH:mm')).head()

#### 添加列，记录开车前30分钟数据

In [ ]:
df_hu_ning2['start_time_before30m'] = df_hu_ning2['start_time'].apply(lambda x: arrow.get(x, 'HH:mm')\
                                                      .replace(minutes=-30).format('HH:mm'))
df_hu_ning2.head()

#### 记录时间==发车前30分钟数据，（高价值数据）

In [ ]:
df_hu_ning2[df_hu_ning2['start_time_before30m'] == df_hu_ning2['record_time']].tail(5)

In [ ]:
df_hu_ning2[(df_hu_ning2['from_station_name'] == u'无锡东') &\
            (df_hu_ning2['station_train_code'] == 'G130') &\
           (df_hu_ning2['record_time'] == '11:34')]

#### df3为可用数据

In [ ]:
df_hu_ning3 = df_hu_ning2[df_hu_ning2['start_time_before30m'] == df_hu_ning2['record_time']]
print len(df_hu_ning3)

In [ ]:
df_hu_ning3[df_hu_ning3['station_train_code'] == 'G130']

#### 加一条列，记录起止站数据

In [ ]:

df_hu_ning3.head(2)

In [ ]:
type(df_hu_ning3['from_to'][934300])

#### from_to为列表

In [ ]:
df_hu_ning3.from_to[934300]

#### 导出为JSON数据

In [ ]:
df_hu_ning3.to_json('./data/df_hu_ning.json')

In [ ]:
df_test = pd.read_json('./data/df_hu_ning.json')

In [ ]:
df_test.head(2)

In [ ]:
df_test.ix[934300,'from_to']

## 2.2 汇总前一区间与下一区间的余票，并计算站点的上下客情况。

>step1:添加一列，统计余票总数（如果有可能，可去获取上车多少人？）;<br>
>step2:前一个数据减后一个数据，即可获得中间站的上下客情况。

>中间站即为下一条数据的from_station_name

# 二、获取各车次区间情况

# 1. 区间站信息获取

## 1.1 获取沪宁-宁沪线车次

https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no=5l000G191200&from_station_telecode=SHH&to_station_telecode=NJH&depart_date=2016-12-23

In [ ]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")

def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                     para, verify=False).text
    return r

def to_csv(js):
    js2 = json.loads(js)
    data = js2['data']['datas']
    df = pd.DataFrame(data)
    df_use = df[[u'train_no',
            u'station_train_code',
                 u'from_station_name',
                 u'to_station_name',
                 u'start_time',
                 u'swz_num',
                 u'tz_num',
                 u'zy_num',
                 u'ze_num',
                 u'wz_num',
                 ]]
    df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    df_use[u'station_train_code'].str.startswith('D')]
    df_use.replace('--',0,inplace=True)
    df_use.replace(u'无',0,inplace=True)
#     tm = arrow.now().format('HH:mm:ss')
#     df_use['record_time'] = tm
    df_use[u'train_no'].to_csv('./data/SH-NJ-train_no.csv',index=False,encoding='utf-8')
#     df_use[u'train_no'].to_csv('./data/NJ-SH-train_no.csv',index=False,encoding='utf-8')
    
if __name__ == '__main__':
    para = 'queryDate=2016-12-29&from_station=SHH&to_station=NJH'
#     para = 'queryDate=2016-12-29&from_station=NJH&to_station=SHH'
    js = get_pd(para)
    to_csv(js)
    print('finished')

## 1.2 获取各车次，站点信息

#### 3个参数需要调整

In [ ]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")


def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no='+para+\
                     '&from_station_telecode=NJH&to_station_telecode=SHH&depart_date=2016-12-29',
                     verify=False).text
    return r

def to_csv(para,js):
    js2 = json.loads(js)
    data = js2['data']['data']
    df = pd.DataFrame(data)
    try:
        df = df[["station_name",]]
        df_use = pd.DataFrame({'train_no':[''],'station_name':['']})
        df_use['train_no'] = para.strip()
        df_use['station_name'] = df_use['station_name'].apply(lambda x: df["station_name"].values)
    #     df_use['start_time'] = df_use['start_time'].apply(lambda x: df["start_time"].values)
    #     df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    #     df_use[u'station_train_code'].str.startswith('D')]
    #     df_use.replace('--',0,inplace=True)
    #     df_use.replace(u'无',0,inplace=True)
    #     tm = arrow.now().format('HH:mm:ss')
    #     df_use['record_time'] = tm
        df_use.to_csv('./data/NJ-SH-station.csv',mode = 'a',index=False,header=False,encoding='utf-8')
    except:
        pass
if __name__ == '__main__':
#     para = '55000G707270'
#     js = get_pd(para)
#     to_csv(para,js)
#     print('finished')
    station_no_file = open('./data/NJ-SH-train_no.csv','r')
    station_no_data = station_no_file.readlines()
    for station_no in station_no_data:
#         print station_no.strip()
#         para = '55000G707270'
        js = get_pd(station_no.strip())
        to_csv(station_no.strip(),js)
    print('finished')

#### 读取信息

In [ ]:
df_SH_NJ = pd.read_csv('./data/SH-NJ-station.csv',encoding='utf-8')
df_SH_NJ.tail()

In [ ]:
len(df_SH_NJ)

In [ ]:
df_SH_NJ.columns=['station_name','train_no']
df_SH_NJ[df_SH_NJ.train_no == '55000G701081']

#### 添加列，进行组合

In [ ]:
print df_SH_NJ['station_name'][0]

In [ ]:
df_SH_NJ['station_name'][0]

In [ ]:
df_SH_NJ['station_name'] = df_SH_NJ['station_name'].apply(lambda x: \
                                              x.replace(u'[',u'').replace(u']',u'').replace(' ','').split(','))

In [ ]:
df_SH_NJ.tail()

In [ ]:
type(df_SH_NJ['station_name'][0])

In [ ]:
len(df_SH_NJ['station_name'][0])

[上海虹桥, 无锡, 常州, 镇江, 南京南, 合肥南, 六安, 金寨, 麻城北, 汉口, 荆州, 宜昌东, 建始, 恩施, 利川, 石柱县, 涪陵北, 重庆北, 潼南, 遂宁, 成都东]

### 示例：将2数进行组合（zip与map函数、join方法的用法）

In [ ]:
seq1 = ['one0', 'one1', 'one2']
seq2 = ['two0', 'two1', 'two2']
zip(seq1, seq2)

In [ ]:
seq = ['one0', 'one1', 'one2', 'two0', 'two1', 'two2']
com= map(list,zip(seq[:-1], seq[1:]))
com

In [ ]:
','.join(com[0])

In [ ]:
my_df['B'] = my_df.A.apply(lambda x: list(map(list, zip(x[:-1], x[1:]))))
my_df

In [ ]:
dm = df_SH_NJ.copy()

In [ ]:
dm['station_interval'] = dm['station_name'].apply(lambda x: list(map(list, zip(x[:-1], x[1:]))))
dm.head()

In [ ]:
dm.station_interval[4]

In [ ]:
type(dm['station_interval'][0][0])

In [ ]:
dm.to_json('./data/df_hu_station_modifity.json')

# 三、余票数据与站点区间数据比对，获取是相邻站点的余票数据信息

In [ ]:
import pandas as pd
import numpy as np
df_tickets = pd.read_json('./data/df_hu_ning.json')
df_tickets[df_tickets['train_no']=='55000G701081']

In [ ]:
len(df_tickets)

In [ ]:
df_tickets.ix[1801584,'from_to']

In [ ]:
df_stations = pd.read_json('./data/df_hu_station_modifity.json')
df_stations[df_stations['train_no']=='55000G701081']

In [ ]:
df_stations.ix[74,'station_interval']

In [ ]:
len(df_stations)

In [ ]:
df_tickets_com = pd.merge(df_tickets,df_stations,on='train_no')
df_tickets_com[df_tickets_com['train_no']=='5l000G192400'].head(2)

In [ ]:
len(df_tickets_com)

In [ ]:
df_tickets_com.to_json('./data/df_tickets_com.json')

In [ ]:
type(df_tickets_com['station_interval'][3][0])

In [ ]:
df_tickets_com.ix[1,'from_to']

```
print (my_df.apply(lambda x: x.A in x.B, axis=1))
0     True
1    False
2     True
dtype: bool
print (my_df[my_df.apply(lambda x: x.A in x.B, axis=1)])
          A                               B
0  [aa, ab]  [[aa, ab], [ba, bb], [bc, bd]]
2  [bc, bd]            [[aa, ab], [bc, bd]]
```

In [ ]:
del df_tickets_com['in']

In [ ]:
df_tickets_com2 = df_tickets_com[df_tickets_com.apply(lambda x: x['from_to'] in x['station_interval'], axis=1)]
df_tickets_com2.head(2)

In [ ]:
len(df_tickets_com2)

In [ ]:
df_tickets_com2.to_json('./df_tickets_to_analysi.json')

# 三、数据分析与数据可视化——以苏州为例

In [ ]:
dff = pd.read_json('./df_tickets_to_analysi.json')
dff.head(2)

In [ ]:
df_to_suzhou = dff[dff['to_station_name'].isin([u'苏州',u'苏州北',u'苏州园区',u'苏州新区'])]
df_to_suzhou.sort_values(by='station_train_code').head(2)

In [ ]:
df_from_suzhou = dff[dff['from_station_name'].isin([u'苏州',u'苏州北',u'苏州园区',u'苏州新区'])]
df_from_suzhou.sort_values(by='station_train_code').head(2)

In [ ]:
df_to_suzhou['tickets_sum_to'] = df_to_suzhou['swz_num']+df_to_suzhou['tz_num']\
+df_to_suzhou['wz_num']+df_to_suzhou['ze_num']+df_to_suzhou['zy_num']
df_to_suzhou.sort_values(by='station_train_code').head(2)

In [ ]:
df_from_suzhou['tickets_sum_from'] = df_from_suzhou['swz_num']+df_from_suzhou['tz_num']\
+df_from_suzhou['wz_num']+df_from_suzhou['ze_num']+df_from_suzhou['zy_num']
df_from_suzhou.sort_values(by='station_train_code').head(2)

In [ ]:
suzhou_df = pd.merge(df_to_suzhou,df_from_suzhou, on = 'train_no')

In [ ]:
suzhou_df.sort_values(by='station_train_code_x').head(2)

In [ ]:
suzhou_df.columns

In [ ]:
suzhou_df['tickets'] = suzhou_df['tickets_sum_to'] - suzhou_df['tickets_sum_from']

In [ ]:
str(suzhou_df['start_time_y'][0])

#### 时间显示转换

In [ ]:
import arrow
suzhou_df['start_time_y'] = suzhou_df['start_time_y'].\
apply(lambda x: arrow.get(str(x),'YY-MM-DD HH:mm:ss').format('HH:mm'))

In [ ]:
suzhou_df_to_plot = suzhou_df[['tickets','start_time_y','train_no','station_train_code_x','station_train_code_y']]

In [ ]:
suzhou_df_to_plot.head(2)

In [ ]:
suzhou_df_to_plot.sort_values(by='start_time_y')

#### 将时间设置为序号

In [ ]:
suzhou_df_to_plot2 = suzhou_df_to_plot.copy()
suzhou_df_to_plot2.head(2)

In [ ]:
suzhou_df_to_plot2.sort_values(by='start_time_y').head(2)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# plt.figure(figsize=(19,7))
dmt= suzhou_df_to_plot2.sort_values(by='start_time_y')
dmt.plot(y='tickets', x ='start_time_y',kind = 'bar',figsize=(14,5), legend=False,fontsize=6)
plt.xlabel(u'时 间(2016-12-21 06:00—23:00)',size=10)
plt.ylabel(u'上下客流量值（单位：人）',size=10)
plt.title(u'苏州地区站点（上海——南京）\n',size=10)
# plt.show()
# plt.tight_layout()
plt.savefig('suzhou.png',dpi=300)

In [ ]:
import arrow
arrow.get('2016-12-19 08:05', 'YY-MM-DD HH:mm').format('HH:mm')

In [ ]:
import arrow
suzhou_df_top_plot.index = suzhou_df_top_plot.index.\
apply(lambda x: arrow.get(x,'YY-MM-DD HH:mm').format('HH:mm'))